In [6]:
SIZE=181
MIN_COUNT=5
WINDOW=6
NEGATIVE_WORDS=6
EPOCH_SIZE=5

# gensim modules
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec

# numpy
import numpy

# shuffle
from random import shuffle

# logging
import logging
import os.path
import sys


import keras
from keras.datasets import mnist
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout,Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, AveragePooling1D
from keras.models import Model
from keras.layers.advanced_activations import LeakyReLU
from keras import backend as K
from keras.utils import np_utils
import numpy as np

program = os.path.basename(sys.argv[0])
logger = logging.getLogger(program)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
logging.root.setLevel(level=logging.INFO)
logger.info("running %s" % ' '.join(sys.argv))

2017-05-01 19:24:19,418 : INFO : running /Users/SamBerndt/miniconda3/envs/deeplearning/lib/python3.5/site-packages/ipykernel_launcher.py -f /Users/SamBerndt/Library/Jupyter/runtime/kernel-9577fc00-d2d4-467a-8c92-63c7f8f739a2.json


In [7]:
class LabeledLineSentence(object):

    def __init__(self, sources):
        self.sources = sources

        flipped = {}

        # make sure that keys are unique
        for key, value in sources.items():
            if value not in flipped:
                flipped[value] = [key]
            else:
                raise Exception('Non-unique prefix encountered')

    def __iter__(self):
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    yield LabeledSentence(utils.to_unicode(line).split(), [prefix + '_%s' % item_no])

    def to_array(self):
        self.sentences = []
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    self.sentences.append(LabeledSentence(
                        utils.to_unicode(line).split(), [prefix + '_%s' % item_no]))
        return self.sentences

    def sentences_perm(self):
        shuffle(self.sentences)
        return self.sentences

In [8]:
for i in range(1,6):
    sources = {'test-neg-'+str(i)+'.txt':'TEST_NEG', 'test-pos-'+str(i)+'.txt':'TEST_POS', 'train-neg-'+str(i)+'.txt':'TRAIN_NEG', 'train-pos-'+str(i)+'.txt':'TRAIN_POS','train-unsup.txt':'TRAIN_UNS'}
    sentences = LabeledLineSentence(sources)
    model = Doc2Vec(min_count=MIN_COUNT, window=WINDOW, size=SIZE, sample=1e-4, negative=5, workers=7)
    model.build_vocab(sentences.to_array())
    for epoch in range(EPOCH_SIZE):
        logger.info('Epoch %d' % epoch)
        model.train(sentences.sentences_perm(),total_examples=model.corpus_count, epochs=model.iter)
    model.save('./problems'+str(i)+'.d2v')



2017-05-01 19:24:19,475 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2017-05-01 19:24:19,493 : INFO : collecting all words and their counts
2017-05-01 19:24:19,500 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-05-01 19:24:19,509 : INFO : collected 3135 word types and 520 unique tags from a corpus of 520 examples and 25416 words
2017-05-01 19:24:19,511 : INFO : Loading a fresh vocabulary
2017-05-01 19:24:19,520 : INFO : min_count=5 retains 953 unique words (30% of original 3135, drops 2182)
2017-05-01 19:24:19,522 : INFO : min_count=5 leaves 21475 word corpus (84% of original 25416, drops 3941)
2017-05-01 19:24:19,528 : INFO : deleting the raw counts dictionary of 3135 items
2017-05-01 19:24:19,530 : INFO : sample=0.0001 downsamples 796 most-common words
2017-05-01 19:24:19,531 : INFO : downsampling leaves estimated 7621 word corpus (35.5% of prior 21475)
2017-05-01 19:24:19,533 : INFO : estimated required memo

2017-05-01 19:24:21,707 : INFO : worker thread finished; awaiting finish of 4 more threads
2017-05-01 19:24:21,711 : INFO : worker thread finished; awaiting finish of 3 more threads
2017-05-01 19:24:21,715 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-05-01 19:24:21,717 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-05-01 19:24:21,723 : INFO : worker thread finished; awaiting finish of 0 more threads
2017-05-01 19:24:21,724 : INFO : training on 127080 raw words (40720 effective words) took 0.3s, 120046 effective words/s
2017-05-01 19:24:21,726 : WARNING : under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay
2017-05-01 19:24:21,727 : INFO : Epoch 1
2017-05-01 19:24:21,729 : INFO : training model with 7 workers on 953 vocabulary and 181 features, using sg=0 hs=0 sample=0.0001 negative=5 window=6
2017-05-01 19:24:21,942 : INFO : worker thread finished; awaiting finish of 6 more threads
2017-05-01 19:

2017-05-01 19:24:23,992 : INFO : worker thread finished; awaiting finish of 0 more threads
2017-05-01 19:24:23,994 : INFO : training on 127080 raw words (40819 effective words) took 0.4s, 115604 effective words/s
2017-05-01 19:24:23,995 : WARNING : under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay
2017-05-01 19:24:23,996 : INFO : Epoch 2
2017-05-01 19:24:23,998 : INFO : training model with 7 workers on 953 vocabulary and 181 features, using sg=0 hs=0 sample=0.0001 negative=5 window=6
2017-05-01 19:24:24,200 : INFO : worker thread finished; awaiting finish of 6 more threads
2017-05-01 19:24:24,345 : INFO : worker thread finished; awaiting finish of 5 more threads
2017-05-01 19:24:24,347 : INFO : worker thread finished; awaiting finish of 4 more threads
2017-05-01 19:24:24,363 : INFO : worker thread finished; awaiting finish of 3 more threads
2017-05-01 19:24:24,366 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-05-01 19:

2017-05-01 19:24:26,280 : INFO : training model with 7 workers on 953 vocabulary and 181 features, using sg=0 hs=0 sample=0.0001 negative=5 window=6
2017-05-01 19:24:26,452 : INFO : worker thread finished; awaiting finish of 6 more threads
2017-05-01 19:24:26,562 : INFO : worker thread finished; awaiting finish of 5 more threads
2017-05-01 19:24:26,567 : INFO : worker thread finished; awaiting finish of 4 more threads
2017-05-01 19:24:26,571 : INFO : worker thread finished; awaiting finish of 3 more threads
2017-05-01 19:24:26,574 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-05-01 19:24:26,578 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-05-01 19:24:26,582 : INFO : worker thread finished; awaiting finish of 0 more threads
2017-05-01 19:24:26,584 : INFO : training on 127080 raw words (40759 effective words) took 0.3s, 140005 effective words/s
2017-05-01 19:24:26,585 : WARNING : under 10 jobs per worker: consider setting a smaller `ba

2017-05-01 19:24:28,620 : INFO : worker thread finished; awaiting finish of 3 more threads
2017-05-01 19:24:28,624 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-05-01 19:24:28,627 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-05-01 19:24:28,629 : INFO : worker thread finished; awaiting finish of 0 more threads
2017-05-01 19:24:28,631 : INFO : training on 127080 raw words (40708 effective words) took 0.3s, 141425 effective words/s
2017-05-01 19:24:28,632 : WARNING : under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay
2017-05-01 19:24:28,634 : INFO : saving Doc2Vec object under ./problems5.d2v, separately None
2017-05-01 19:24:28,636 : INFO : not storing attribute syn0norm
2017-05-01 19:24:28,638 : INFO : not storing attribute cum_table
2017-05-01 19:24:28,666 : INFO : saved ./problems5.d2v


In [11]:
Result=[]
for i in range(1,2):
    print(i)
    model = Doc2Vec.load('./problems'+str(i)+'.d2v')
    logger.info('Sentiment')
    train_arrays = numpy.zeros((168+173, SIZE))
    train_labels = numpy.zeros(168+173)

    for i in range(168):
        prefix_train_pos = 'TRAIN_POS_' + str(i)
        train_arrays[i] = model.docvecs[prefix_train_pos]
        train_labels[i] = 1
    for i in range(173):
        prefix_train_neg = 'TRAIN_NEG_' + str(i)
        train_arrays[168 + i] = model.docvecs[prefix_train_neg]
        train_labels[168 + i] = 0
    
    test_arrays = numpy.zeros((42+42, SIZE))
    test_labels = numpy.zeros(42+42)

    for i in range(42):
        prefix_test_pos = 'TEST_POS_' + str(i)
        test_arrays[i] = model.docvecs[prefix_test_pos]
        test_labels[i] = 1
    for i in range(42):
        prefix_test_neg = 'TEST_NEG_' + str(i)
        test_arrays[42 + i] = model.docvecs[prefix_test_neg]
        test_labels[42 + i] = 0 
    #from __future__ import print_function
    batch_size = 16
    num_classes = 2
    epochs = 128

    # input image dimensions
    img_rows, img_cols = 28, 28

    # # the data, shuffled and split between train and test sets
    # (x_train, y_train), (x_test, y_test) = mnist.load_data()

    # if K.image_data_format() == 'channels_first':
    #     x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    #     x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    #     input_shape = (1, img_rows, img_cols)
    # else:
    #     x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    #     x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    #     input_shape = (img_rows, img_cols, 1)
    x_train=train_arrays.reshape(train_arrays.shape[0], SIZE,1)
    x_test=test_arrays.reshape(test_arrays.shape[0], SIZE,1)
    input_shape = (SIZE)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    #print('x_train shape:', x_train.shape)
    #print(x_train.shape[0], 'train samples')
    #print(x_test.shape[0], 'test samples')
    y_train=np_utils.to_categorical(train_labels, 2)
    y_test=np_utils.to_categorical(test_labels, 2)
    #print(y_test)
    num_classes=2
    sequence_input = (SIZE,1)
    Kmodel = Sequential()
    Kmodel.add(Conv1D(128, kernel_size=2,activation='relu',input_shape=sequence_input))
    
    #print(Kmodel.output_shape)
    #Kmodel.add(AveragePooling1D(3))
    Kmodel.add(Conv1D(64, 2, activation='relu'))
    Kmodel.add(Dropout(.5))
    #print(Kmodel.output_shape)
    Kmodel.add(Flatten())
    #print(Kmodel.output_shape)
    Kmodel.add(Dense(256, activation='relu'))
    Kmodel.add(Dropout(.5))
    #print(Kmodel.output_shape)
    Kmodel.add(Dense(num_classes, activation='softmax'))
    #print(Kmodel.output_shape)
    sgd = optimizers.SGD(lr=0.1, decay=1e-5, momentum=0.5, nesterov=True)
    Kmodel.compile(optimizer=sgd,loss='binary_crossentropy',metrics=['accuracy'])
    Kmodel.summary()
    #Kmodel.fit(x_train, y_train,
              # batch_size=batch_size,
               #epochs=epochs,
               #verbose=1)
    #score = Kmodel.evaluate(x_test, y_test, verbose=0) 
    '''
    from sklearn.linear_model import LogisticRegression

    logger.info('Fitting')
    classifier = LogisticRegression()
    classifier.fit(train_arrays, train_labels)

    LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
              intercept_scaling=1, penalty='l2', random_state=None, tol=0.0001)
    '''
    Result.append(classifier.score(test_arrays, test_labels))
print(Result)


2017-05-01 19:25:53,950 : INFO : loading Doc2Vec object from ./problems1.d2v
2017-05-01 19:25:53,969 : INFO : loading wv recursively from ./problems1.d2v.wv.* with mmap=None
2017-05-01 19:25:53,970 : INFO : setting ignored attribute syn0norm to None
2017-05-01 19:25:53,971 : INFO : loading docvecs recursively from ./problems1.d2v.docvecs.* with mmap=None
2017-05-01 19:25:53,972 : INFO : setting ignored attribute cum_table to None
2017-05-01 19:25:53,973 : INFO : loaded ./problems1.d2v
2017-05-01 19:25:53,979 : INFO : Sentiment


1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 180, 128)          384       
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 179, 64)           16448     
_________________________________________________________________
dropout_3 (Dropout)          (None, 179, 64)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 11456)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               2932992   
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 514       
Total pa

In [27]:
Result

[0.66666666666666663,
 0.73809523809523814,
 0.80952380952380953,
 0.77380952380952384,
 0.76190476190476186]

In [5]:
Result

[0.69047618479955764,
 0.73809523525692167,
 0.8690476162093026,
 0.76190476758139469,
 0.77380952380952384]

In [6]:
for epoch in range(EPOCH_SIZE):
    logger.info('Epoch %d' % epoch)
    model.train(sentences.sentences_perm())

2017-04-19 13:50:01,892 : INFO : Epoch 0
2017-04-19 13:50:01,894 : INFO : training model with 7 workers on 849 vocabulary and 181 features, using sg=0 hs=0 sample=0.0001 negative=5 window=6
2017-04-19 13:50:01,895 : INFO : expecting 445 sentences, matching count from corpus used for vocabulary survey
2017-04-19 13:50:01,972 : INFO : worker thread finished; awaiting finish of 6 more threads
2017-04-19 13:50:01,976 : INFO : worker thread finished; awaiting finish of 5 more threads
2017-04-19 13:50:01,979 : INFO : worker thread finished; awaiting finish of 4 more threads
2017-04-19 13:50:02,003 : INFO : worker thread finished; awaiting finish of 3 more threads
2017-04-19 13:50:02,005 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-04-19 13:50:02,006 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-04-19 13:50:02,007 : INFO : worker thread finished; awaiting finish of 0 more threads
2017-04-19 13:50:02,008 : INFO : training on 108830 raw words

2017-04-19 13:50:02,696 : WARNING : under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay
2017-04-19 13:50:02,697 : INFO : Epoch 7
2017-04-19 13:50:02,699 : INFO : training model with 7 workers on 849 vocabulary and 181 features, using sg=0 hs=0 sample=0.0001 negative=5 window=6
2017-04-19 13:50:02,700 : INFO : expecting 445 sentences, matching count from corpus used for vocabulary survey
2017-04-19 13:50:02,769 : INFO : worker thread finished; awaiting finish of 6 more threads
2017-04-19 13:50:02,770 : INFO : worker thread finished; awaiting finish of 5 more threads
2017-04-19 13:50:02,772 : INFO : worker thread finished; awaiting finish of 4 more threads
2017-04-19 13:50:02,799 : INFO : worker thread finished; awaiting finish of 3 more threads
2017-04-19 13:50:02,800 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-04-19 13:50:02,801 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-04-19 13:50:02,802 

2017-04-19 13:50:04,055 : INFO : training on 108830 raw words (32640 effective words) took 0.2s, 155594 effective words/s
2017-04-19 13:50:04,057 : WARNING : under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay
2017-04-19 13:50:04,059 : INFO : Epoch 14
2017-04-19 13:50:04,061 : INFO : training model with 7 workers on 849 vocabulary and 181 features, using sg=0 hs=0 sample=0.0001 negative=5 window=6
2017-04-19 13:50:04,063 : INFO : expecting 445 sentences, matching count from corpus used for vocabulary survey
2017-04-19 13:50:04,212 : INFO : worker thread finished; awaiting finish of 6 more threads
2017-04-19 13:50:04,214 : INFO : worker thread finished; awaiting finish of 5 more threads
2017-04-19 13:50:04,220 : INFO : worker thread finished; awaiting finish of 4 more threads
2017-04-19 13:50:04,278 : INFO : worker thread finished; awaiting finish of 3 more threads
2017-04-19 13:50:04,281 : INFO : worker thread finished; awaiting finish of 2 more 

2017-04-19 13:50:05,644 : INFO : worker thread finished; awaiting finish of 0 more threads
2017-04-19 13:50:05,645 : INFO : training on 108830 raw words (32478 effective words) took 0.2s, 163230 effective words/s
2017-04-19 13:50:05,646 : WARNING : under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay
2017-04-19 13:50:05,647 : INFO : Epoch 21
2017-04-19 13:50:05,649 : INFO : training model with 7 workers on 849 vocabulary and 181 features, using sg=0 hs=0 sample=0.0001 negative=5 window=6
2017-04-19 13:50:05,650 : INFO : expecting 445 sentences, matching count from corpus used for vocabulary survey
2017-04-19 13:50:05,723 : INFO : worker thread finished; awaiting finish of 6 more threads
2017-04-19 13:50:05,725 : INFO : worker thread finished; awaiting finish of 5 more threads
2017-04-19 13:50:05,727 : INFO : worker thread finished; awaiting finish of 4 more threads
2017-04-19 13:50:05,756 : INFO : worker thread finished; awaiting finish of 3 more 

2017-04-19 13:50:06,885 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-04-19 13:50:06,887 : INFO : worker thread finished; awaiting finish of 0 more threads
2017-04-19 13:50:06,888 : INFO : training on 108830 raw words (32436 effective words) took 0.2s, 159469 effective words/s
2017-04-19 13:50:06,890 : WARNING : under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay
2017-04-19 13:50:06,892 : INFO : Epoch 28
2017-04-19 13:50:06,894 : INFO : training model with 7 workers on 849 vocabulary and 181 features, using sg=0 hs=0 sample=0.0001 negative=5 window=6
2017-04-19 13:50:06,895 : INFO : expecting 445 sentences, matching count from corpus used for vocabulary survey
2017-04-19 13:50:07,043 : INFO : worker thread finished; awaiting finish of 6 more threads
2017-04-19 13:50:07,045 : INFO : worker thread finished; awaiting finish of 5 more threads
2017-04-19 13:50:07,048 : INFO : worker thread finished; awaiting finish of 4 more 

In [7]:
model.save('./problems.d2v')


2017-04-19 13:50:07,233 : INFO : saving Doc2Vec object under ./problems.d2v, separately None
2017-04-19 13:50:07,235 : INFO : not storing attribute syn0norm
2017-04-19 13:50:07,237 : INFO : not storing attribute cum_table
2017-04-19 13:50:07,258 : INFO : saved ./problems.d2v


In [8]:
model = Doc2Vec.load('./problems.d2v')

2017-04-19 13:50:07,264 : INFO : loading Doc2Vec object from ./problems.d2v
2017-04-19 13:50:07,283 : INFO : loading wv recursively from ./problems.d2v.wv.* with mmap=None
2017-04-19 13:50:07,284 : INFO : setting ignored attribute syn0norm to None
2017-04-19 13:50:07,286 : INFO : loading docvecs recursively from ./problems.d2v.docvecs.* with mmap=None
2017-04-19 13:50:07,288 : INFO : setting ignored attribute cum_table to None
2017-04-19 13:50:07,289 : INFO : loaded ./problems.d2v


In [9]:
model.most_similar('tree')

2017-04-19 13:50:07,298 : INFO : precomputing L2-norms of word weight vectors


[('binary', 0.7200572490692139),
 ('traversal', 0.6357672214508057),
 ('root', 0.619725227355957),
 ('null', 0.5980157852172852),
 ('parent', 0.5489043593406677),
 ('iteratively', 0.5446182489395142),
 ('serialization', 0.5315271615982056),
 ('node', 0.5144803524017334),
 ('depth', 0.5037304162979126),
 ('rooted', 0.46393850445747375)]

In [10]:
model.docvecs['TRAIN_POS_0']

array([-0.42700383,  0.15686961, -0.81781024,  0.33888963, -0.4351117 ,
       -0.60683674,  0.37969643, -0.39887372,  0.20718117,  0.18954435,
        0.20413658,  0.08190788, -0.00186256, -0.06489961, -0.38141572,
        0.13288324, -0.25746876, -0.12187643, -0.49393845, -0.284675  ,
       -0.60127342, -0.02445807, -0.03931512, -0.34321567,  0.00510921,
       -0.33436996, -0.46393687,  0.97205985, -0.42188919,  0.08270755,
        0.55625206, -0.29406157, -0.36597544,  0.26222777,  0.19877712,
        0.37964451, -0.02299048, -0.17546292,  0.68391132,  0.26711851,
       -0.60058987, -0.42773843, -0.61160451,  0.21487713,  0.17754468,
       -0.04207525,  0.15950716, -0.67377335,  0.0900462 , -0.21861014,
       -0.28650326,  0.72860634,  0.56776065,  0.23104177, -0.08413443,
        0.03368078,  0.09681886, -0.32686681,  0.29843473,  0.01053979,
       -0.40292317,  0.32523409, -0.39770582, -0.40455988,  0.48213717,
        0.45164061, -0.66728586, -0.53140426,  0.12764584,  0.09

In [11]:
model.docvecs['TEST_POS_0']

array([ 0.33878735, -0.01870725,  0.61508656,  0.21386501, -0.13169347,
       -0.0424075 ,  0.14809611,  0.76269025,  0.37332204,  0.00409013,
       -0.8097629 ,  0.42468837, -0.41287485,  0.42817107, -0.15906805,
       -0.0881356 ,  0.5488742 , -0.63306284, -0.799851  ,  0.16910644,
        0.16366991,  0.23343833, -0.53457409, -0.87418908, -0.84722573,
        0.6326859 , -0.84055418,  1.3634727 , -0.25016397,  0.29088408,
       -0.04470953,  0.67132831, -1.0402081 , -0.08928592,  0.11422954,
       -1.11452365, -0.46962801,  0.52722901, -0.02901716,  0.50797582,
        0.60929602, -0.01770802,  0.61513662,  0.02591623, -0.04418965,
        0.10245298,  0.00757933, -0.62416345,  0.69201964,  0.5481779 ,
        0.29058671,  1.15814781,  0.16065739,  0.60190052,  0.43543732,
       -0.93699157,  0.33294255,  0.16765498, -0.07827894,  0.10369367,
        0.15580198, -0.64003444, -0.19600624, -0.56780094, -0.5991568 ,
        0.46954256, -0.39748475,  0.79050368, -0.62915969,  0.11

In [12]:
logger.info('Sentiment')
train_arrays = numpy.zeros((210+154, SIZE))
train_labels = numpy.zeros(210+154)

for i in range(210):
    prefix_train_pos = 'TRAIN_POS_' + str(i)
    train_arrays[i] = model.docvecs[prefix_train_pos]
    train_labels[i] = 1
for i in range(154):
    prefix_train_neg = 'TRAIN_NEG_' + str(i)
    train_arrays[210 + i] = model.docvecs[prefix_train_neg]
    train_labels[210 + i] = 0

2017-04-19 13:50:07,394 : INFO : Sentiment


KeyError: 'TRAIN_POS_170'

In [ ]:
test_arrays = numpy.zeros((52+38, SIZE))
test_labels = numpy.zeros(52+38)

for i in range(52):
    prefix_test_pos = 'TEST_POS_' + str(i)
    test_arrays[i] = model.docvecs[prefix_test_pos]
    test_labels[i] = 1
for i in range(38):
    prefix_test_neg = 'TEST_NEG_' + str(i)
    test_arrays[52 + i] = model.docvecs[prefix_test_neg]
    test_labels[52 + i] = 0


In [ ]:
from sklearn.linear_model import LogisticRegression

logger.info('Fitting')
classifier = LogisticRegression()
classifier.fit(train_arrays, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, penalty='l2', random_state=None, tol=0.0001)

print(classifier.score(test_arrays, test_labels))


In [ ]:
from sklearn import svm
classifier = svm.SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=True, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
classifier.fit(train_arrays, train_labels)
print(classifier.score(test_arrays,test_labels))

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout,Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, AveragePooling1D
from keras.models import Model
from keras.layers.advanced_activations import LeakyReLU
from keras import backend as K
from keras.utils import np_utils
import numpy as np
batch_size = 20
num_classes = 2
epochs = 15

# input image dimensions
img_rows, img_cols = 28, 28

# # the data, shuffled and split between train and test sets
# (x_train, y_train), (x_test, y_test) = mnist.load_data()

# if K.image_data_format() == 'channels_first':
#     x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
#     x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
#     input_shape = (1, img_rows, img_cols)
# else:
#     x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
#     x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
#     input_shape = (img_rows, img_cols, 1)
x_train=train_arrays.reshape(train_arrays.shape[0], SIZE,1)
x_test=test_arrays.reshape(test_arrays.shape[0], SIZE,1)
input_shape = (SIZE)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = 255
x_test = 255
#print('x_train shape:', x_train.shape)
#print(x_train.shape[0], 'train samples')
#print(x_test.shape[0], 'test samples')
y_train=np_utils.to_categorical(train_labels, 2)
y_test=np_utils.to_categorical(test_labels, 2)
#print(y_test)
num_classes=2
sequence_input = (SIZE,1)
Kmodel = Sequential()
Kmodel.add(Conv1D(64, kernel_size=1,activation='linear',input_shape=sequence_input))
Kmodel.add(LeakyReLU(alpha=.001))
#print(Kmodel.output_shape)
Kmodel.add(AveragePooling1D(3))
#Kmodel.add(Conv1D(128, 2, activation='linear'))
#Kmodel.add(LeakyReLU(alpha=.001)) 
#Kmodel.add(AveragePooling1D())
#print(Kmodel.output_shape)
Kmodel.add(Flatten())
#print(Kmodel.output_shape)
Kmodel.add(Dense(128, activation='linear'))
Kmodel.add(LeakyReLU(alpha=.001))
#print(Kmodel.output_shape)
Kmodel.add(Dense(num_classes, activation='softmax'))
#print(Kmodel.output_shape)
Kmodel.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
Kmodel.fit(x_train, y_train,
           batch_size=batch_size,
           epochs=epochs,
           verbose=1)
score = Kmodel.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
Kmodel.metrics_names

import numpy as np
np.random.seed(123)  # for reproducibility
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, normalization
from keras.utils import np_utils
from keras.datasets import mnist, cifar10
from keras.layers.advanced_activations import LeakyReLU
from sklearn.decomposition import PCA


# Basic Prepocess function for MNIST
# Uses PCA and Normalization
def pca_mnist(X, X_test):
    X = X.astype('float32') / 255.
    X_test = X_test.astype('float32') / 255.
    X = X.reshape((len(X), np.prod(X.shape[1:])))
    X_test = X_test.reshape((len(X_test), np.prod(X_test.shape[1:])))
    print(X.shape)
    pca = PCA(n_components=100)
    pca.fit(X)
    X = pca.inverse_transform(pca.transform(X))
    X_test = pca.inverse_transform(pca.transform(X_test))
    X = X.reshape(X.shape[0], 1, 28, 28)
    X_test = X_test.reshape(X_test.shape[0], 1, 28, 28)
    print(X.shape)
    return X, X_test

#Preprocess for cifar10
#Doesn't really affect performance
def pca_cifar_pre(X):
    X2 = np.transpose(X, (3, 0, 1, 2))
    temp1 = 0
    temp2 = 0
    temp3 = 0
    for i in range(3): #individual PCA for each color 
        temp_X = X2[i][:][:][:]
        temp_X = np.transpose(temp_X, (0, 2, 1))
        temp_X = pca_cifar(temp_X)
        if i == 0:
            temp1 = temp_X
        elif i == 1:
            temp2 = temp_X
        else:
            temp3 = temp_X
    X_temp = np.array([temp1, temp2, temp3])
    # print(X.shape)
    X = np.transpose(X, (1, 2, 3, 0))
    X = X.astype('float32') / 255.
    X = X.reshape(X.shape[0], 3, 32, 32)
    return X


def pca_cifar(X):
    X = X.astype('float32') / 255.
    X = X.reshape((len(X), np.prod(X.shape[1:])))
    print(X.shape)
    pca = PCA(n_components=1024)
    pca.fit(X)
    X = pca.inverse_transform(pca.transform(X))
    X = X.reshape(X.shape[0], 32, 32)
    print(X.shape)
    return X


(X_train, y_train), (X_test, y_test) = mnist.load_data()



# 6. Preprocess class labels
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

# 7. Define model architecture
model = Sequential()

model.add(Convolution(64, kernel_size=(5, 5), data_format="channels_first", activation='relu',input_shape=(1, 28, 28)))


print(model.output_shape)
model.add(Convolution2D(64,kernel_size=(5, 5), activation='linear')) #Convolutinal Leaky Layer
model.add(LeakyReLU(alpha=.001))
print(model.output_shape)
model.add(MaxPooling2D(pool_size=(2, 2))) #Vanila Pooling
print(model.output_shape)
model.add(Dropout(0.25))

model.add(Flatten())
print(model.output_shape)
model.add(Dense(128, activation='relu'))
print(model.output_shape)
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))


model.summary

#Compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    
#Fit model on training data
model.fit(X_train, Y_train, batch_size=32, epochs=20, verbose=1)


#Evaluate model on test data
score = model.evaluate(X_test, Y_test, verbose=1)
print(score)
